## Tâche 177 - Créer un modele DBT pour les résultats PFAS

### Contexte :

Partir de `select * from int__resultats_udi_communes` (cf https://data-for-good.slack.com/archives/C08B329AG7M/p1742063272779779)

Nous voulons calculer les résultats/situations pour les PFAS pour :

- le dernier relevé par UDI (Prio 1)
- par année par UDI (Prio 2)
- pour le dernier relevé par commune (Prio 3)
- par année par commune (Prio 3)

6 situations:

- Pas recherché
- Aucun paramètre n’a été quantifié
- Au moins 1 paramètre a été quantifié mais la somme des 20 PFAS < 0,1 µg/L et la somme des 4 PFAS < 0,02 µg/L
- Somme des 20 PFAS < 0,1 µg/L et la somme des 4 PFAS (PFOA, PFOS, PFNA, PFHXS) > 0.02 µg/L
- Somme des 20 PFAS > 0,1 µg/L (fait passer l’affichage total polluant en orange)
- Au moins 1 PFAS > valeur sanitaire (fait passer l’affichage total polluant en rouge)

(pour le bilan annuel, aller lire le brief)

### Choses à faire :

Créer un modèle dbt pour "resultat pfas udi", et un autre modèle "resultat pfas commune"

Ecrire le code SQL qui produit le résultat table avec ces colonnes :

- cdreseau (dans le cas d'UDI, sinon inseecommune)
- periode ("dernier relevé" ou l'année)
- resultat : le résultat / la situation
- date : quand c'est le dernier relévé, rajouter le datetimeprel

Trouver un exemple de commune dans chaque cas pour illustrer avec les prélèvements bruts associés


In [71]:
%reload_ext sql
%sql duckdb:///../../database/data.duckdb
%config SqlMagic.displaylimit = 20

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

Ci-dessous, le code de Vinca pour les derniers résultats CVM par UDI :


In [45]:
%%sql

WITH
list_communes_uid AS (
    SELECT DISTINCT cdreseau
    FROM
        int__lien_commune_cdreseau
),

last_pvl AS (
    SELECT
        *,
        ROW_NUMBER()
            OVER (
                PARTITION BY cdreseau, cdparametresiseeaux
                ORDER BY datetimeprel DESC
            )
            AS row_number
    FROM
        int__resultats_udi_communes
    WHERE
        categorie = 'cvm' -- à supprimer pour avoir pour tout
),

communes_last_pvl AS (
    SELECT last_pvl.* EXCLUDE (last_pvl.row_number)
    FROM
        last_pvl
    WHERE
        last_pvl.row_number = 1
),

resultat_communes_last_pvl AS (
    SELECT
        cdreseau,
        cdparametresiseeaux,
        datetimeprel,
        CASE
            WHEN valtraduite = 0 THEN 'jamais quantifié'
            WHEN valtraduite = 1 THEN 'jamais quantifié'
            WHEN
                limitequal_float IS NOT NULL AND valtraduite > limitequal_float
                THEN '> 0,5 µg/L'
            WHEN
                limitequal_float IS NOT NULL AND valtraduite <= limitequal_float
                THEN '<= 0,5 µg/L'
            ELSE 'error check SQL'
        END AS resultat
    FROM
        communes_last_pvl
),

resultats_all_communes_last AS (
    SELECT
        list_communes_uid.cdreseau,
        'cvm' AS categorie,
        resultat_communes_last_pvl.cdparametresiseeaux,
        resultat_communes_last_pvl.datetimeprel AS last_datetimeprel,
        COALESCE(resultat_communes_last_pvl.resultat, 'Pas recherché')
            AS resultat
    FROM
        list_communes_uid
    LEFT JOIN
        resultat_communes_last_pvl
        ON
            list_communes_uid.cdreseau = resultat_communes_last_pvl.cdreseau
)

SELECT
    resultats_all_communes_last.*,
    'dernier prélévement' AS periode
FROM
    resultats_all_communes_last
LIMIT 10

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdreseau,categorie,cdparametresiseeaux,last_datetimeprel,resultat,periode
027000369,cvm,CLVYL,2024-07-11 09:30:00,jamais quantifié,dernier prélévement
028001344,cvm,CLVYL,2024-12-16 09:00:00,"<= 0,5 µg/L",dernier prélévement
028001209,cvm,CLVYL,2025-01-10 10:23:00,jamais quantifié,dernier prélévement
029000352,cvm,CLVYL,2025-01-02 11:30:00,jamais quantifié,dernier prélévement
02A001278,cvm,CLVYL,2024-10-08 09:20:00,jamais quantifié,dernier prélévement
02A000540,cvm,CLVYL,2021-06-22 08:45:00,jamais quantifié,dernier prélévement
02B001824,cvm,CLVYL,2021-05-03 06:45:00,jamais quantifié,dernier prélévement
030000560,cvm,CLVYL,2024-07-31 10:09:00,jamais quantifié,dernier prélévement
030000156,cvm,CLVYL,2024-11-26 10:55:00,"<= 0,5 µg/L",dernier prélévement
030000651,cvm,CLVYL,2024-10-22 08:55:00,jamais quantifié,dernier prélévement


In [72]:
%%sql

SELECT *
FROM int__resultats_udi_communes
WHERE categorie = 'pfas'

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

referenceprel,cdparametresiseeaux,valtraduite,limitequal,de_partition,limitequal_float,unite,categorie,cdreseau,inseecommune,datetimeprel
00700187109,PFBA,0.000,None,2022,None,None,pfas,007001817,07039,2022-09-14 09:10:00
00700187109,PFBS,0.000,None,2022,None,None,pfas,007001817,07039,2022-09-14 09:10:00
00700187109,PFDA,0.000,None,2022,None,None,pfas,007001817,07039,2022-09-14 09:10:00
00700187109,PFDODA,0.000,None,2022,None,None,pfas,007001817,07039,2022-09-14 09:10:00
00700187109,PFDODS,0.000,None,2022,None,None,pfas,007001817,07039,2022-09-14 09:10:00
00700187109,PFDS,0.000,None,2022,None,None,pfas,007001817,07039,2022-09-14 09:10:00
00700187109,PFHPA,0.010,None,2022,None,None,pfas,007001817,07039,2022-09-14 09:10:00
00700187109,PFHPS,0.000,None,2022,None,None,pfas,007001817,07039,2022-09-14 09:10:00
00700187109,PFHXA,0.022,None,2022,None,None,pfas,007001817,07039,2022-09-14 09:10:00
00700187109,PFHXS,0.000,None,2022,None,None,pfas,007001817,07039,2022-09-14 09:10:00


In [28]:
%%sql

SELECT DISTINCT libmajparametre, cdparametre, cdparametresiseeaux
FROM stg_edc__resultats
WHERE libmajparametre LIKE 'SOMME DE % PFAS'
GROUP BY 1, 2, 3

Running query in 'duckdb:///../../database/data.duckdb'

libmajparametre,cdparametre,cdparametresiseeaux
SOMME DE 20 PFAS,8847,SPFAS


Il va falloir créer un CTE qui regroupe le groupe des 4 PFAS (PFOA, PFOS, PFNA, PFHXS). On veut les colonnes suivantes : cdparametre, cdparametresiseeaux


In [48]:
%%sql

SELECT DISTINCT cdparametre, cdparametresiseeaux
FROM stg_edc__resultats
WHERE cdparametresiseeaux IN ('PFOA', 'PFOS', 'PFNA', 'PFHXS')
GROUP BY 1, 2

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

cdparametre,cdparametresiseeaux
5347,PFOA
6508,PFNA
6561,PFOS
6830,PFHXS


In [74]:
%%sql

WITH latest_pfas_results AS (
    SELECT
        *,
        ROW_NUMBER()
            OVER (
                PARTITION BY cdreseau, cdparametresiseeaux
                ORDER BY datetimeprel DESC
            ) AS row_number
    FROM int__resultats_udi_communes
    WHERE categorie = 'pfas'
)

SELECT 
    referenceprel, 
    cdreseau, 
    datetimeprel,
    SUM(CASE WHEN cdparametresiseeaux IN ('PFOA', 'PFOS', 'PFNA', 'PFHXS') THEN valtraduite ELSE 0 END) AS somme_4_pfas
FROM latest_pfas_results
WHERE row_number = 1
GROUP BY referenceprel, cdreseau, datetimeprel
ORDER BY somme_4_pfas DESC

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

referenceprel,cdreseau,datetimeprel,somme_4_pfas
02A00113033,02A000841,2023-04-24 09:45:00,10.000
01300273898,013001457,2025-01-21 09:40:00,0.396
03400327431,034000985,2025-01-15 14:24:00,0.112
01100172447,011004114,2025-01-24 11:05:00,0.097
01100172447,011000329,2025-01-24 11:05:00,0.097
01300273795,013000416,2025-01-21 08:48:00,0.085
03400327430,034001188,2025-01-13 14:44:00,0.064
08900130113,089000574,2024-03-22 10:47:00,0.061
03900122750,039001435,2024-09-04 09:28:00,0.058
01800095087,018000371,2024-11-14 10:21:00,0.057


In [ ]:
%%sql

SELECT *
FROM int__resultats_udi_communes
WHERE referenceprel = '02A00113033' AND cdreseau = '02A000841' AND categorie = 'pfas'

Running query in 'duckdb:///../../database/data.duckdb'

referenceprel,cdparametresiseeaux,valtraduite,limitequal,de_partition,limitequal_float,unite,categorie,cdreseau,inseecommune,datetimeprel
02A00113033,PFNA,10.000,None,2023,None,None,pfas,02A000841,2A276,2023-04-24 09:45:00


In [83]:
%%sql --save last_results_pfas_udi

WITH latest_pfas_results AS (
    SELECT
        *,
        ROW_NUMBER() OVER (
            PARTITION BY cdreseau, cdparametresiseeaux
            ORDER BY datetimeprel DESC
        ) AS row_number
    FROM int__resultats_udi_communes
    WHERE categorie = 'pfas'
),
aggregated_results AS (
    SELECT
        referenceprel,
        cdreseau,
        datetimeprel,
        MAX(CASE WHEN cdparametresiseeaux = 'SPFAS' THEN valtraduite ELSE 0 END) AS sum_20_pfas,
        SUM(CASE WHEN cdparametresiseeaux IN ('PFOA', 'PFOS', 'PFNA', 'PFHXS') THEN valtraduite ELSE 0 END) AS sum_4_pfas,
        MAX(CASE WHEN cdparametresiseeaux != 'SPFAS' THEN valtraduite ELSE NULL END) AS max_individual_pfas,
        COUNT(DISTINCT CASE WHEN valtraduite NOT IN (0, 1) THEN cdparametresiseeaux END) AS nb_quantified_params
    FROM latest_pfas_results
    WHERE row_number = 1
    GROUP BY referenceprel, cdreseau, datetimeprel
)
SELECT
    referenceprel,
    cdreseau,
    datetimeprel,
    CASE
        WHEN sum_20_pfas IS NULL AND nb_quantified_params = 0 THEN 'Aucun paramètre n’a été quantifié'
        WHEN sum_20_pfas < 0.1 AND sum_4_pfas < 0.02 THEN 'Somme des 20 PFAS < 0,1 µg/L et somme des 4 PFAS < 0,02 µg/L'
        WHEN sum_20_pfas < 0.1 AND sum_4_pfas >= 0.02 THEN 'Somme des 20 PFAS < 0,1 µg/L et somme des 4 PFAS >= 0,02 µg/L'
        WHEN sum_20_pfas >= 0.1 THEN 'Somme des 20 PFAS >= 0,1 µg/L'
        WHEN max_individual_pfas >= 0.1 THEN 'Au moins 1 PFAS >= valeur sanitaire (0,1 µg/L)'
        ELSE 'Erreur de classification'
    END AS result
FROM aggregated_results
ORDER BY datetimeprel DESC

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

referenceprel,cdreseau,datetimeprel,result
02600171893,026001154,2025-01-31 12:23:00,"Somme des 20 PFAS < 0,1 µg/L et somme des 4 PFAS < 0,02 µg/L"
00500134428,005001232,2025-01-31 11:17:00,"Somme des 20 PFAS < 0,1 µg/L et somme des 4 PFAS < 0,02 µg/L"
04500164331,045000430,2025-01-31 10:02:00,"Somme des 20 PFAS < 0,1 µg/L et somme des 4 PFAS < 0,02 µg/L"
04500164331,045000431,2025-01-31 10:02:00,"Somme des 20 PFAS < 0,1 µg/L et somme des 4 PFAS < 0,02 µg/L"
04500164334,045000494,2025-01-31 09:38:00,"Somme des 20 PFAS < 0,1 µg/L et somme des 4 PFAS < 0,02 µg/L"
03800342072,038000488,2025-01-31 09:15:00,"Somme des 20 PFAS < 0,1 µg/L et somme des 4 PFAS < 0,02 µg/L"
04400263448,044000151,2025-01-31 09:01:00,"Somme des 20 PFAS < 0,1 µg/L et somme des 4 PFAS < 0,02 µg/L"
04400263448,044000155,2025-01-31 09:01:00,"Somme des 20 PFAS < 0,1 µg/L et somme des 4 PFAS < 0,02 µg/L"
04400263448,044000652,2025-01-31 09:01:00,"Somme des 20 PFAS < 0,1 µg/L et somme des 4 PFAS < 0,02 µg/L"
04400263448,044003468,2025-01-31 09:01:00,"Somme des 20 PFAS < 0,1 µg/L et somme des 4 PFAS < 0,02 µg/L"


In [84]:
%%sql --with last_results_pfas_udi

SELECT MIN(referenceprel) AS referenceprel_test, MIN(cdreseau) AS cdreseau_test, result
FROM last_results_pfas_udi
GROUP BY result
ORDER BY result DESC

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

referenceprel_test,cdreseau_test,result
01100172447,011000329,"Somme des 20 PFAS >= 0,1 µg/L"
00100142949,001000404,"Somme des 20 PFAS < 0,1 µg/L et somme des 4 PFAS >= 0,02 µg/L"
00100139638,001000356,"Somme des 20 PFAS < 0,1 µg/L et somme des 4 PFAS < 0,02 µg/L"
07200133081,072000498,Erreur de classification


In [87]:
%%sql --with last_results_pfas_udi

SELECT *
FROM last_results_pfas_udi
WHERE result = 'Erreur de classification'

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

referenceprel,cdreseau,datetimeprel,result
07200133081,072000498,2023-04-14 10:39:00,Erreur de classification
07200133081,072000924,2023-04-14 10:39:00,Erreur de classification


In [ ]:
%%sql

SELECT *
FROM int__resultats_udi_communes
WHERE referenceprel = '07200133081' AND cdreseau = '072000498' AND categorie = 'pfas' --AND cdparametresiseeaux IN ('PFOA', 'PFOS', 'PFNA', 'PFHXS')
ORDER BY inseecommune ASC

Running query in 'duckdb:///../../database/data.duckdb'

referenceprel,cdparametresiseeaux,valtraduite,limitequal,de_partition,limitequal_float,unite,categorie,cdreseau,inseecommune,datetimeprel
07200133081,PFOA,None,None,2023,None,None,pfas,072000498,72012,2023-04-14 10:39:00
07200133081,PFOA,None,None,2023,None,None,pfas,072000498,72152,2023-04-14 10:39:00
07200133081,PFOA,None,None,2023,None,None,pfas,072000498,72297,2023-04-14 10:39:00
07200133081,PFOA,None,None,2023,None,None,pfas,072000498,72332,2023-04-14 10:39:00
07200133081,PFOA,None,None,2023,None,None,pfas,072000498,72362,2023-04-14 10:39:00
07200133081,PFOA,None,None,2023,None,None,pfas,072000498,72370,2023-04-14 10:39:00
